# 임베딩 층 사용하기

문장의 긍정, 부정을 판단하는 감성 분류 모델을 만들어 보자

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
# 문장과 레이블 데이터 생성. 긍정은 1, 부정은 0
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

In [3]:
# 케라스의 Tokenizer()를 이용하여 토큰화 시킨다.
t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1

print(vocab_size)

16


In [4]:
# 각 문장에 대해서 정수 인코딩
X_encoded = t.texts_to_sequences(sentences)
print(X_encoded)

[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


In [5]:
max_len=max(len(l) for l in X_encoded)
print(max_len)

4


문장들 중에서 가장 긴 문장의 길이는 4이다. 그래서 모든 문장을 패딩하여 길이를 4로 만들어준다.

In [6]:
X_train=pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train=np.array(y_train)
print(X_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len)) # 모든 임베딩 벡터는 4차원.
model.add(Flatten()) # Dense의 입력으로 넣기위함.
model.add(Dense(1, activation='sigmoid'))

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 0.6942 - acc: 0.4286
Epoch 2/100
1/1 - 0s - loss: 0.6927 - acc: 0.5714
Epoch 3/100
1/1 - 0s - loss: 0.6913 - acc: 0.5714
Epoch 4/100
1/1 - 0s - loss: 0.6898 - acc: 0.5714
Epoch 5/100
1/1 - 0s - loss: 0.6884 - acc: 0.5714
Epoch 6/100
1/1 - 0s - loss: 0.6869 - acc: 0.5714
Epoch 7/100
1/1 - 0s - loss: 0.6855 - acc: 0.5714
Epoch 8/100
1/1 - 0s - loss: 0.6840 - acc: 0.5714
Epoch 9/100
1/1 - 0s - loss: 0.6826 - acc: 0.5714
Epoch 10/100
1/1 - 0s - loss: 0.6811 - acc: 0.5714
Epoch 11/100
1/1 - 0s - loss: 0.6797 - acc: 0.5714
Epoch 12/100
1/1 - 0s - loss: 0.6782 - acc: 0.5714
Epoch 13/100
1/1 - 0s - loss: 0.6768 - acc: 0.5714
Epoch 14/100
1/1 - 0s - loss: 0.6753 - acc: 0.5714
Epoch 15/100
1/1 - 0s - loss: 0.6739 - acc: 0.5714
Epoch 16/100
1/1 - 0s - loss: 0.6724 - acc: 0.5714
Epoch 17/100
1/1 - 0s - loss: 0.6709 - acc: 0.5714
Epoch 18/100
1/1 - 0s - loss: 0.6695 - acc: 0.5714
Epoch 19/100
1/1 - 0s - loss: 0.6680 - acc: 0.5714
Epoch 20/100
1/1 - 0s - loss: 0.6666 - a

# 사전 훈련된 워드 임베딩 사용하기

사전 훈련된 GloVe와 Word2Vec 임베딩을 사용해서 모델을 훈련시키는 실습을 해보자

GloVe 다운로드 링크 : http://nlp.stanford.edu/data/glove.6B.zip   
Word2Vec 다운로드 링크 : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM

In [11]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip


--2021-02-25 04:56:20--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
접속 nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... 접속됨.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-02-25 04:56:21--  https://nlp.stanford.edu/data/glove.6B.zip
접속 nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... 접속됨.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-02-25 04:56:22--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
접속 downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... 접속됨.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  1.

In [12]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [9]:
print(X_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [10]:
print(y_train)

[1 0 0 1 1 0 1]


## 1) 사전 훈련된 GloVe 사용하기

In [15]:
n=0
f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split() # 각 줄을 읽어와서 word_vector에 저장.
    print("word_vector : ", word_vector) # 각 줄을 출력
    word = word_vector[0] # word_vector에서 첫번째 값만 저장
    print("word : ", word) # word_vector의 첫번째 값만 출력
    n=n+1
    if n==2: # 2개의 줄만 읽어오도록 설정
        break
f.close()

word_vector :  ['the', '-0.038194', '-0.24487', '0.72812', '-0.39961', '0.083172', '0.043953', '-0.39141', '0.3344', '-0.57545', '0.087459', '0.28787', '-0.06731', '0.30906', '-0.26384', '-0.13231', '-0.20757', '0.33395', '-0.33848', '-0.31743', '-0.48336', '0.1464', '-0.37304', '0.34577', '0.052041', '0.44946', '-0.46971', '0.02628', '-0.54155', '-0.15518', '-0.14107', '-0.039722', '0.28277', '0.14393', '0.23464', '-0.31021', '0.086173', '0.20397', '0.52624', '0.17164', '-0.082378', '-0.71787', '-0.41531', '0.20335', '-0.12763', '0.41367', '0.55187', '0.57908', '-0.33477', '-0.36559', '-0.54857', '-0.062892', '0.26584', '0.30205', '0.99775', '-0.80481', '-3.0243', '0.01254', '-0.36942', '2.2167', '0.72201', '-0.24978', '0.92136', '0.034514', '0.46745', '1.1079', '-0.19358', '-0.074575', '0.23353', '-0.052062', '-0.22044', '0.057162', '-0.15806', '-0.30798', '-0.41625', '0.37972', '0.15006', '-0.53212', '-0.2055', '-1.2526', '0.071624', '0.70565', '0.49744', '-0.42063', '0.26148', '-1.

In [16]:
print(type(word_vector))
print(len(word_vector))

<class 'list'>
101


word_vector안 101개 값 중에서 첫번째 값은 임베딩 벡터가 의미하는 단어를 말하고, 두번째 값부터 101번째 값까지는 해당 단어의 임베딩 벡터 100개 차원에서의 각 값을 말한다. 즉 'glove.6B.100d.txt'파일은 많은 단어에 대해서 100개의 차원을 가지는 임베딩 벡터로 제공하고 있다.

이제 모든 임베딩 벡터들을 불러오기 위해서 형식은 파이썬의 사전형 구조를 사용하도록 한다.

In [17]:
import numpy as np
embedding_dict = dict()
f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

400000개의 Embedding vector가 있습니다.


In [18]:
# 임의의 단어 'respectable'에 대한 임베딩 벡터값과 임베딩 벡터 차원 출력

print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

In [19]:
embedding_matrix = np.zeros((vocab_size, 100))
# 단어 집합 크기의 행과 100개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

(16, 100)

In [20]:
print(t.word_index.items())

dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


In [21]:
for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어를 1개씩 꺼내온다.
    temp = embedding_dict.get(word) # 단어(key) 해당되는 임베딩 벡터의 100개의 값(value)를 임시 변수에 저장
    if temp is not None:
        embedding_matrix[i] = temp # 임수 변수의 값을 단어와 맵핑되는 인덱스의 행에 삽입

이제 훈련 데이터의 단어 집합의 모든 단어에 대하여 사전 훈련된 GloVe의 임베딩 벡터들을 맵핑하였다. 이를 이용하여 임베딩 층을 만들어 본다.

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)

현재 실습에서 사전 훈련된 워드 임베딩 차원이 100인 값을 사용하므로 출력 차원을 100으로 하고,사전 훈련된 임베딩을 그대로 사용할 것이기 때문에 훈련을 더 이상 하지 않게 하기 위해서 trainable을 False로 설정한다.

In [23]:
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.8857 - acc: 0.2857
Epoch 2/100
1/1 - 0s - loss: 0.8640 - acc: 0.2857
Epoch 3/100
1/1 - 0s - loss: 0.8429 - acc: 0.2857
Epoch 4/100
1/1 - 0s - loss: 0.8225 - acc: 0.2857
Epoch 5/100
1/1 - 0s - loss: 0.8027 - acc: 0.4286
Epoch 6/100
1/1 - 0s - loss: 0.7835 - acc: 0.4286
Epoch 7/100
1/1 - 0s - loss: 0.7648 - acc: 0.4286
Epoch 8/100
1/1 - 0s - loss: 0.7466 - acc: 0.5714
Epoch 9/100
1/1 - 0s - loss: 0.7290 - acc: 0.5714
Epoch 10/100
1/1 - 0s - loss: 0.7118 - acc: 0.5714
Epoch 11/100
1/1 - 0s - loss: 0.6952 - acc: 0.5714
Epoch 12/100
1/1 - 0s - loss: 0.6790 - acc: 0.5714
Epoch 13/100
1/1 - 0s - loss: 0.6632 - acc: 0.5714
Epoch 14/100
1/1 - 0s - loss: 0.6479 - acc: 0.5714
Epoch 15/100
1/1 - 0s - loss: 0.6330 - acc: 0.5714
Epoch 16/100
1/1 - 0s - loss: 0.6184 - acc: 0.7143
Epoch 17/100
1/1 - 0s - loss: 0.6043 - acc: 0.7143
Epoch 18/100
1/1 - 0s - loss: 0.5905 - acc: 0.7143
Epoch 19/100
1/1 - 0s - loss: 0.5770 - acc: 0.8571
Epoch 20/100
1/1 - 0s - loss: 0.5639 - a

## 2) 사전 훈련된 Word2Vec 사용하기

In [24]:
import numpy as np
import gensim

In [25]:
# 현재 위치에 구글의 사전 훈련된 Word2Vec을 다운로드
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-02-25 05:39:49--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.240.214
접속 s3.amazonaws.com (s3.amazonaws.com)|52.216.240.214|:443... 접속됨.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz.1’

GoogleNews-vectors- 100%[===================>]   1.53G  13.6MB/s    in 2m 50s  

2021-02-25 05:42:41 (9.24 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz.1’ saved [1647046227/1647046227]



In [26]:
# 구글의 사전 훈련된 Word2vec 모델을 로드합니다.
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [27]:
print(word2vec_model.vectors.shape) # 모델의 크기 확인

(3000000, 300)


이 모델의 단어 총 개수는 3000000개이고 임베딩 차원이 300인 것을 확인할 수 있다.

In [28]:
embedding_matrix = np.zeros((vocab_size, 300))
# 단어 집합 크기의 행과 300개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

(16, 300)

In [29]:
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [30]:
for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어와 정수 인덱스를 1개씩 꺼내온다.
    temp = get_vector(word) # 단어(key) 해당되는 임베딩 벡터의 300개의 값(value)를 임시 변수에 저장
    if temp is not None: # 만약 None이 아니라면 임베딩 벡터의 값을 리턴받은 것이므로
        embedding_matrix[i] = temp # 해당 단어 위치의 행에 벡터의 값을 저장한다.

In [31]:
print(word2vec_model['nice'])

[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
 -0.01806641 -0.25390625  0.13183594  0.0859375   0.16113281  0.11083984
 -0.11083984 -0.0859375   0.0267334   0.34570312  0.15136719 -0.00415039
  0.10498047  0.04907227 -0.06982422  0.08642578  0.03198242 -0.02844238
 -0.15722656  0.11865234  0.36132812  0.00173187  0.05297852 -0.234375
  0.11767578  0.08642578 -0.01123047  0.25976562  0.28515625 -0.11669922
  0.38476562  0.07275391  0.01147461  0.03466797  0.18164062 -0.03955078
  0.04199219  0.01013184 -0.06054688  0.09765625  0.06689453  0.14648438
 -0.12011719  0.08447266 -0.06152344  0.06347656  0.3046875  -0.35546875
 -0.2890625   0.19628906 -0.33203125 -0.07128906  0.12792969  0.09619141
 -0.12158203 -0.08691406 -0.12890625  0.27734375  0.265625    0.1796875
  0.12695312  0.06298828 -0.34375    -0.05908203  0.0456543   0.171875
  0.08935547  0.14648438 -0.04638672 -0.00842285 -0.0279

In [32]:
print('단어 nice의 정수 인덱스 :', t.word_index['nice'])

단어 nice의 정수 인덱스 : 1


In [33]:
print(embedding_matrix[1]) # 단어 'nice'의 임베딩 벡터값 출력

[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
 -0.01806641 -0.25390625  0.13183594  0.0859375   0.16113281  0.11083984
 -0.11083984 -0.0859375   0.0267334   0.34570312  0.15136719 -0.00415039
  0.10498047  0.04907227 -0.06982422  0.08642578  0.03198242 -0.02844238
 -0.15722656  0.11865234  0.36132812  0.00173187  0.05297852 -0.234375
  0.11767578  0.08642578 -0.01123047  0.25976562  0.28515625 -0.11669922
  0.38476562  0.07275391  0.01147461  0.03466797  0.18164062 -0.03955078
  0.04199219  0.01013184 -0.06054688  0.09765625  0.06689453  0.14648438
 -0.12011719  0.08447266 -0.06152344  0.06347656  0.3046875  -0.35546875
 -0.2890625   0.19628906 -0.33203125 -0.07128906  0.12792969  0.09619141
 -0.12158203 -0.08691406 -0.12890625  0.27734375  0.265625    0.1796875
  0.12695312  0.06298828 -0.34375    -0.05908203  0.0456543   0.171875
  0.08935547  0.14648438 -0.04638672 -0.00842285 -0.0279

이제 Embedding에 사전 훈련된 embeddinmg_matrix를 입력으로 넣어주고 모델을 학습시켜본다.

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 0.6580 - acc: 0.7143
Epoch 2/100
1/1 - 0s - loss: 0.6408 - acc: 0.7143
Epoch 3/100
1/1 - 0s - loss: 0.6240 - acc: 0.7143
Epoch 4/100
1/1 - 0s - loss: 0.6078 - acc: 0.7143
Epoch 5/100
1/1 - 0s - loss: 0.5921 - acc: 0.8571
Epoch 6/100
1/1 - 0s - loss: 0.5768 - acc: 1.0000
Epoch 7/100
1/1 - 0s - loss: 0.5621 - acc: 1.0000
Epoch 8/100
1/1 - 0s - loss: 0.5478 - acc: 1.0000
Epoch 9/100
1/1 - 0s - loss: 0.5340 - acc: 1.0000
Epoch 10/100
1/1 - 0s - loss: 0.5207 - acc: 1.0000
Epoch 11/100
1/1 - 0s - loss: 0.5078 - acc: 1.0000
Epoch 12/100
1/1 - 0s - loss: 0.4953 - acc: 1.0000
Epoch 13/100
1/1 - 0s - loss: 0.4832 - acc: 1.0000
Epoch 14/100
1/1 - 0s - loss: 0.4715 - acc: 1.0000
Epoch 15/100
1/1 - 0s - loss: 0.4602 - acc: 1.0000
Epoch 16/100
1/1 - 0s - loss: 0.4493 - acc: 1.0000
Epoch 17/100
1/1 - 0s - loss: 0.4387 - acc: 1.0000
Epoch 18/100
1/1 - 0s - loss: 0.4284 - acc: 1.0000
Epoch 19/100
1/1 - 0s - loss: 0.4185 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.4089 - a